<a href="https://colab.research.google.com/github/TomPelletreauDuris/Subjectivity_mining_Assigment_3/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 7.4 MB/s 
     |████████████████████████████████| 1.9 MB 65.7 MB/s 
     |████████████████████████████████| 9.2 MB 52.9 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 1.3 MB 56.3 MB/s 
     |████████████████████████████████| 5.3 MB 39.4 MB/s 
     |████████████████████████████████| 441 kB 73.7 MB/s 
     |████████████████████████████████| 7.6 MB 32.5 MB/s 
     |████████████████████████████████| 163 kB 72.6 MB/s 
     |████████████████████████████████| 182 kB 66.3 MB/s 
     |████████████████████████████████| 166 kB 72.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 72.4 MB/s 
     |████████████████████████████████| 162 kB 44.9 MB/s 
     |████████████████████████████████| 162 kB 72.8 MB/s 
     |██████████████████████████████

#Assignment 4 

###Experimental Setup


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hasoc-train.csv to hasoc-train.csv
Saving olid-test.csv to olid-test.csv
Saving olid-train.csv to olid-train.csv
Saving olid-train-small.csv to olid-train-small.csv


In [ ]:
import pandas as pd
import io
 
df1 = pd.read_csv(io.BytesIO(uploaded['olid-train-small.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded['olid-test.csv']))
df3 = pd.read_csv(io.BytesIO(uploaded['hasoc-train.csv']))
df4 = pd.read_csv(io.BytesIO(uploaded['olid-train.csv']))
print(df1)
print(df2)
print(df3)
#df_val is the rest of the training data, which will be used for tuning hyperparameters
df_val = pd.concat([df1,df4]).drop_duplicates(keep=False)


         id                                               text  labels
0     24590  @USER @USER @USER @USER @USER Hahah a left tel...       0
1     19287  @USER @USER I’m glad you do babe (I kiss you b...       0
2     44676          @USER And I have concerns with”Democrats”       0
3     96110                             @USER AS IT SHOULD BE!       0
4     51557  @USER @USER A horrendous act of course. Conser...       1
...     ...                                                ...     ...
5847  24288  @USER This information is out there and should...       1
5848  13930  #MorningJoe the closer it gets to monday the l...       0
5849  68792  @USER @USER Long time prisoners will also have...       0
5850  19909  @USER @USER It wasn’t that long ago it was tab...       0
5851  33705                    @USER He is so beautiful there.       0

[5852 rows x 3 columns]
        id                                               text  labels
0    15923  #WhoIsQ #WheresTheServer #DumpNike #DECLA

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs


###Prelimnary analysis 

Analysis of the BERT hyperparameters.

In [ ]:
#We start to set some hyperparameters as the learning rate and the number of epochs to measure differences
model_args_1 = ClassificationArgs()
model_args_2 = ClassificationArgs()
model_args_3 = ClassificationArgs()
model_args_4 = ClassificationArgs()
model_args_5 = ClassificationArgs()
model_args_6 = ClassificationArgs()

#Learning rate
model_args_1.learning_rate = 1e-5
model_args_2.learning_rate = 5e-5
model_args_3.learning_rate = 1e-4

#Authorizing the overwriting
model_args_1.overwrite_output_dir = True
model_args_2.overwrite_output_dir = True
model_args_3.overwrite_output_dir = True

#We start to implement the learning rate 1st to select the best one and then try to change the number of epochs
model_Bert_1 = ClassificationModel(
    "bert", "bert-base-cased", args=model_args_1
)
model_Bert_2 = ClassificationModel(
    "bert", "bert-base-cased", args=model_args_2
)
model_Bert_3 = ClassificationModel(
    "bert", "bert-base-cased", args=model_args_3
)

#We train the different models one by one
model_Bert_1.train_model(df1, output_dir="Bert ID Training 1")
model_Bert_2.train_model(df1, output_dir="Bert ID Training 2")
model_Bert_3.train_model(df1, output_dir="Bert ID Training 3")

#We evaluate the different models to compare their results and choose the best paramater
result_Bert_1, model_outputs_Bert_1, wrong_predictions_Bert_1 = model_Bert_1.eval_model(df_val)
result_Bert_2, model_outputs_Bert_2, wrong_predictions_Bert_2 = model_Bert_2.eval_model(df_val)
result_Bert_3, model_outputs_Bert_3, wrong_predictions_Bert_3 = model_Bert_3.eval_model(df_val)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

  0%|          | 0/7388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

  0%|          | 0/7388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

In [ ]:
wrong_predictions_Bert_1

[{'guid': 2, 'text_a': '@USER Liberals are all Kookoo !!!', 'text_b': None, 'label': 1},
 {'guid': 4, 'text_a': '@USER was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism', 'text_b': None, 'label': 1},
 {'guid': 7, 'text_a': '@USER @USER @USER gun control! That is all these kids are asking for!', 'text_b': None, 'label': 1},
 {'guid': 9, 'text_a': '@USER @USER Kind of like when conservatives wanna associate everyone to their left as communist antifa members?', 'text_b': None, 'label': 1},
 {'guid': 11, 'text_a': "@USER Tbh these days i just don't like people in general i just don't connect with people these days just a annoyance..", 'text_b': None, 'label': 1},
 {'guid': 13, 'text_a': 'The only thing the Democrats have is lying and stalling to stop Trump from being #President.  What have they done for you lately. #Trump #Kavanaugh #MAGA #DEMSUCK', 'text_b': None, 'labe

In [ ]:
result_Bert_1, model_outputs_Bert_1, wrong_predictions_Bert_1 = model_Bert_1.eval_model(df2)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
#We display the results and calculate the F1 score to compare them
F1_Bert_1 = 2*((result_Bert_1["tp"] / (result_Bert_1["tp"] + result_Bert_1["fp"])) * (result_Bert_1["tp"]/(result_Bert_1["tp"]+result_Bert_1["fn"]))) / ((result_Bert_1["tp"] / (result_Bert_1["tp"] + result_Bert_1["fp"])) + (result_Bert_1["tp"]/(result_Bert_1["tp"]+result_Bert_1["fn"])))
F1_Bert_2 = 2*((result_Bert_2["tp"] / (result_Bert_2["tp"] + result_Bert_2["fp"])) * (result_Bert_2["tp"]/(result_Bert_2["tp"]+result_Bert_2["fn"]))) / ((result_Bert_2["tp"] / (result_Bert_2["tp"] + result_Bert_2["fp"])) + (result_Bert_2["tp"]/(result_Bert_2["tp"]+result_Bert_2["fn"])))
F1_Bert_3 = 2*((result_Bert_3["tp"] / (result_Bert_3["tp"] + result_Bert_3["fp"])) * (result_Bert_3["tp"]/(result_Bert_3["tp"]+result_Bert_3["fn"]))) / ((result_Bert_3["tp"] / (result_Bert_3["tp"] + result_Bert_3["fp"])) + (result_Bert_3["tp"]/(result_Bert_3["tp"]+result_Bert_3["fn"])))


In [ ]:
print(F1_Bert_1)
print(F1_Bert_2)
print(F1_Bert_3)

0.650613154960981
0.6383790226460071
0.5923486192044592


In [ ]:
#Nb of epochs with the most efficient learning rate
model_args_4.num_train_epochs = 1
model_args_4.learning_rate = 1e-5
model_args_4.overwrite_output_dir = True

model_args_5.num_train_epochs = 2
model_args_5.learning_rate = 1e-5
model_args_5.overwrite_output_dir = True

model_args_6.num_train_epochs = 5
model_args_6.learning_rate = 1e-5
model_args_6.overwrite_output_dir = True

#We implement the models
model_Bert_4 = ClassificationModel(
    "bert", "bert-base-cased", args=model_args_4
)
model_Bert_5 = ClassificationModel(
    "bert", "bert-base-cased", args=model_args_5
)
model_Bert_6 = ClassificationModel(
    "bert", "bert-base-cased", args=model_args_6
)

#We train the different models one by one
model_Bert_4.train_model(df1, output_dir="Bert ID Training 4")
model_Bert_5.train_model(df1, output_dir="Bert ID Training 5")
model_Bert_6.train_model(df1, output_dir="Bert ID Training 6")

#We evaluate the different models to compare their results and choose the best paramater
result_Bert_4, model_outputs_Bert_4, wrong_predictions_Bert_4 = model_Bert_4.eval_model(df_val)
result_Bert_5, model_outputs_Bert_5, wrong_predictions_Bert_5 = model_Bert_5.eval_model(df_val)
result_Bert_6, model_outputs_Bert_6, wrong_predictions_Bert_6 = model_Bert_6.eval_model(df_val)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

  0%|          | 0/5852 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

  0%|          | 0/7388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

  0%|          | 0/7388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

In [ ]:
#We display the 4 result and calculate the F1 score to compare them
F1_Bert_4 = 2*((result_Bert_4["tp"] / (result_Bert_4["tp"] + result_Bert_4["fp"])) * (result_Bert_4["tp"]/(result_Bert_4["tp"]+result_Bert_4["fn"]))) / ((result_Bert_4["tp"] / (result_Bert_4["tp"] + result_Bert_4["fp"])) + (result_Bert_4["tp"]/(result_Bert_4["tp"]+result_Bert_4["fn"])))
F1_Bert_5 = 2*((result_Bert_5["tp"] / (result_Bert_5["tp"] + result_Bert_5["fp"])) * (result_Bert_5["tp"]/(result_Bert_5["tp"]+result_Bert_5["fn"]))) / ((result_Bert_5["tp"] / (result_Bert_5["tp"] + result_Bert_5["fp"])) + (result_Bert_5["tp"]/(result_Bert_5["tp"]+result_Bert_5["fn"])))
F1_Bert_6 = 2*((result_Bert_6["tp"] / (result_Bert_6["tp"] + result_Bert_6["fp"])) * (result_Bert_6["tp"]/(result_Bert_6["tp"]+result_Bert_6["fn"]))) / ((result_Bert_6["tp"] / (result_Bert_6["tp"] + result_Bert_6["fp"])) + (result_Bert_6["tp"]/(result_Bert_6["tp"]+result_Bert_6["fn"])))

In [ ]:
print(F1_Bert_4)
print(F1_Bert_5)
print(F1_Bert_6)

0.6521348314606741
0.6588606169781581
0.645604986030518


###In-domain experiments : 
After having chosen good hyperparameters, we decided to train the different models with these very same we train the BERT (pre-trained language model), RoBERTa (pre-trained language model) and HateBERT (re-trained transformer model) on the OLID Train dataset (olid-train-small.csv) and evaluate them on the OLID test set (olid_test.csv)

In [ ]:
#We assumed that these preliminary results about the Bert model could apply to most of the others 
model_args_Roberta = ClassificationArgs()

#Learning rate
model_args_Roberta.learning_rate = 1e-5
#Nb of epochs
model_args_Roberta.num_train_epochs = 1
#Authorization of overwriting
model_args_Roberta.overwrite_output_dir = True

#implementation
model_Roberta = ClassificationModel(
    "roberta", "roberta-base", args=model_args_Roberta
)

#training
model_Roberta.train_model(df1, output_dir="Roberta ID Training")

#We evaluate the model
result_Roberta, model_outputs_Roberta, wrong_predictions_Roberta = model_Roberta.eval_model(df2)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
result_Roberta, model_outputs_Roberta, wrong_predictions_Roberta = model_Roberta.eval_model(df2)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
#the results of Roberta
F1_Roberta = 2*((result_Roberta["tp"] / (result_Roberta["tp"] + result_Roberta["fp"])) * (result_Roberta["tp"]/(result_Roberta["tp"]+result_Roberta["fn"]))) / ((result_Roberta["tp"] / (result_Roberta["tp"] + result_Roberta["fp"])) + (result_Roberta["tp"]/(result_Roberta["tp"]+result_Roberta["fn"])))
print(F1_Roberta)

0.70020964360587


In [ ]:
#We assumed that these preliminary results about the Bert model could apply to most of the others 
model_args_HateBert = ClassificationArgs()

#Learning rate
model_args_HateBert.learning_rate = 1e-5
#Nb of epochs
model_args_HateBert.num_train_epochs = 5
#Authorization of overwriting
model_args_HateBert.overwrite_output_dir = True

#implementation
model_HateBert = ClassificationModel(
    "bert", "GroNLP/hateBERT", args=model_args_HateBert
)

#training
model_HateBert.train_model(df1, output_dir="HateBert ID Training")

#We evaluate the model
result_HateBert, model_outputs_HateBert, wrong_predictions_HateBert = model_HateBert.eval_model(df2)


Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly

Downloading:   0%|          | 0.00/151 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Running Epoch 2 of 5:   0%|          | 0/732 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/732 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/732 [00:00<?, ?it/s]

  0%|          | 0/7388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/924 [00:00<?, ?it/s]

In [ ]:
result_HateBert, model_outputs_HateBert, wrong_predictions_HateBert = model_HateBert.eval_model(df2)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
#the results of HateBert
F1_HateBert = 2*((result_HateBert["tp"] / (result_HateBert["tp"] + result_HateBert["fp"])) * (result_HateBert["tp"]/(result_HateBert["tp"]+result_HateBert["fn"]))) / ((result_HateBert["tp"] / (result_HateBert["tp"] + result_HateBert["fp"])) + (result_HateBert["tp"]/(result_HateBert["tp"]+result_HateBert["fn"])))
print(F1_HateBert)

0.6515463917525772


In [ ]:
max([1,2])

2

###The results :


In [ ]:
print ("BERT MODEL : ")
print(model_Bert_4)
print("precision : ")
print((result_Bert_4["tp"] / (result_Bert_4["tp"] + result_Bert_4["fp"])))
print("recall : ") 
print((result_Bert_4["tp"]/(result_Bert_4["tp"]+result_Bert_4["fn"])))
F1_Bert_4

BERT MODEL : 
precision : 
0.6278667243617482
recall : 
0.678354371201496


0.6521348314606741

In [ ]:
print ("RoBERTa MODEL : ")
print(result_Roberta)
print((result_Roberta["tp"] / (result_Roberta["tp"] + result_Roberta["fp"])))
print((result_Roberta["tp"]/(result_Roberta["tp"]+result_Roberta["fn"])))
F1_Roberta

RoBERTa MODEL : 
{'mcc': 0.585193853427191, 'tp': 167, 'tn': 550, 'fp': 70, 'fn': 73, 'auroc': 0.8803057795698925, 'auprc': 0.7534478246568692, 'eval_loss': 0.3910616062305592}
0.7046413502109705
0.6958333333333333


0.70020964360587

In [ ]:
print ("HateBERT MODEL : ")
print(result_HateBert)
print((result_HateBert["tp"] / (result_HateBert["tp"] + result_HateBert["fp"])))
print((result_HateBert["tp"]/(result_HateBert["tp"]+result_HateBert["fn"])))
F1_HateBert

HateBERT MODEL : 
{'mcc': 0.5147775994401639, 'tp': 158, 'tn': 533, 'fp': 87, 'fn': 82, 'auroc': 0.8464448924731183, 'auprc': 0.7138241524657909, 'eval_loss': 0.6667169619489599}
0.6448979591836734
0.6583333333333333


0.6515463917525772

In [ ]:
wrong_predictions_HateBert

[{'guid': 2, 'text_a': '@USER Liberals are all Kookoo !!!', 'text_b': None, 'label': 1},
 {'guid': 5, 'text_a': '@USER Canada doesn’t need another CUCK! We already have enough #LooneyLeft #Liberals f**king up our great country! #Qproofs #TrudeauMustGo', 'text_b': None, 'label': 1},
 {'guid': 9, 'text_a': '@USER @USER Kind of like when conservatives wanna associate everyone to their left as communist antifa members?', 'text_b': None, 'label': 1},
 {'guid': 11, 'text_a': "@USER Tbh these days i just don't like people in general i just don't connect with people these days just a annoyance..", 'text_b': None, 'label': 1},
 {'guid': 12, 'text_a': '@USER @USER @USER She?  To whom are you referring to?  Hillary?  You know what is tiresome?  Bernie supporters bashing Hillary Clinton.  She is an actual Democrat who raises money for the Democratic Party and doesn’t bash them.  She has a positive message.  So who is dividing the left?', 'text_b': None, 'label': 1},
 {'guid': 14, 'text_a': '@USER 

###Ensemble Methods

In [ ]:
model_outputs_Bert_1
model_outputs_Roberta
model_outputs_HateBert

bert_prediction = []
hatebert_prediction = []
roberta_prediction = []

for x in model_outputs_Bert_1:
  if x[0] > x[1]:
    bert_prediction.append(0)
  else:
    bert_prediction.append(1)

for x in model_outputs_Roberta:
  if x[0] > x[1]:
    roberta_prediction.append(0)
  else:
    roberta_prediction.append(1)

for x in model_outputs_HateBert:
  if x[0] > x[1]:
    hatebert_prediction.append(0)
  else:
    hatebert_prediction.append(1)
print(model_outputs_Bert_1.shape)
print(model_outputs_Roberta.shape)
print(model_outputs_HateBert.shape)

(860, 2)
(860, 2)
(860, 2)


In [ ]:
zero_labels = []
one_labels = []
soft_prediction = []

for x in range(len(model_outputs_Bert_1)):
  current_zero = [model_outputs_Bert_1[x][0],model_outputs_Roberta[x][0],model_outputs_HateBert[x][0]]
  current_one = [model_outputs_Bert_1[x][1],model_outputs_Roberta[x][1],model_outputs_HateBert[x][1]]

  zero_labels.append(max(current_zero))
  one_labels.append(max(current_one))

  if zero_labels[x] > one_labels[x]:
    soft_prediction.append(0)
  else:
    soft_prediction.append(1)

soft_prediction
len(soft_prediction)

860

In [ ]:
model_outputs_Bert_1.shape
len(df2['text'])

860

In [ ]:
import numpy as np

ensemble_df = pd.DataFrame(columns=['id','Tweet','Bert_probability','RoBERTa_probability','HateBERT_probability','Bert_prediction','RoBERTa_prediction','HateBERT_prediction','Label', 'Hard Label', 'Soft Label'])
ensemble_df['Tweet'] = df2['text']
ensemble_df['id'] = df2['id']
ensemble_df['Label'] = df2['labels']
ensemble_df['Bert_probability'] = model_outputs_Bert_1
ensemble_df['RoBERTa_probability'] = model_outputs_Roberta
ensemble_df['HateBERT_probability'] = model_outputs_HateBert
ensemble_df['Bert_prediction'] = bert_prediction
ensemble_df['RoBERTa_prediction'] = roberta_prediction
ensemble_df['HateBERT_prediction'] = hatebert_prediction
ensemble_df['Hard Label'] = np.round((ensemble_df['Bert_prediction'] + ensemble_df['RoBERTa_prediction'] + ensemble_df['HateBERT_prediction'])/3)
ensemble_df['Soft Label'] = soft_prediction
ensemble_df

,id,Tweet,Bert_probability,RoBERTa_probability,HateBERT_probability,Bert_prediction,RoBERTa_prediction,HateBERT_prediction,Label,Hard Label,Soft Label
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,-0.130493,-0.461914,0.752441,1,1,0,1,1.0,0
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0.048279,0.760742,-1.056641,0,0,1,0,0.0,1
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0.744629,0.840820,1.027344,0,0,0,0,0.0,0
3,13876,#Watching #Boomer getting the news that she is...,1.120117,1.160156,2.625000,0,0,0,0,0.0,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,0.936035,0.771973,2.511719,0,0,0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,0.078430,0.099609,1.161133,0,0,0,1,0.0,0
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,1.055664,1.245117,2.554688,0,0,0,0,0.0,0
857,67018,3 people just unfollowed me for talking about ...,-0.519531,-0.781250,0.987793,1,1,0,1,1.0,0
858,50665,#WednesdayWisdom Antifa calls the right fascis...,0.175171,-0.080078,-1.442383,0,1,1,0,1.0,1


In [ ]:
from sklearn.metrics import classification_report
print("Hard label")
print(classification_report(ensemble_df['Hard Label'] ,ensemble_df['Label'] ))
print("Soft label")
print(classification_report(ensemble_df['Soft Label'] ,ensemble_df['Label'] ))


Hard label
              precision    recall  f1-score   support

         0.0       0.91      0.88      0.89       639
         1.0       0.68      0.74      0.71       221

    accuracy                           0.84       860
   macro avg       0.79      0.81      0.80       860
weighted avg       0.85      0.84      0.85       860

Soft label
              precision    recall  f1-score   support

           0       0.86      0.88      0.87       612
           1       0.68      0.66      0.67       248

    accuracy                           0.81       860
   macro avg       0.77      0.77      0.77       860
weighted avg       0.81      0.81      0.81       860



In [ ]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=38c7584c2a9d576f5f82480e54b38624aa0bcc690c4add34652267ff87c05dc3
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


IN-DOMAIN STACKING ENSEMBLE

In [ ]:
from sklearn.model_selection import StratifiedKFold 
import numpy as np
#5FCV output for ensemble 

model_bert_args = ClassificationArgs()
model_roberta_args = ClassificationArgs()
model_hatebert_args = ClassificationArgs()

#Args bert
model_bert_args.learning_rate = 1e-5
model_bert_args.num_train_epochs = 1
model_bert_args.overwrite_output_dir = True

#Args roberta
model_roberta_args.learning_rate = 1e-5
model_roberta_args.num_train_epochs = 1
model_roberta_args.overwrite_output_dir = True

#Args hatebert
model_hatebert_args.learning_rate = 1e-5
model_hatebert_args.num_train_epochs = 1
model_hatebert_args.overwrite_output_dir = True

train = df1

rskf = StratifiedKFold(n_splits=5, shuffle=True)

pred_bert = []
pred_roberta = []
pred_hatebert = []

idx = []
gold = []
text = []

for train_index, test_index in rskf.split(train['text'], train['labels']):

  train_df = train.iloc[train_index]
  test_df = train.iloc[test_index]

  idx.extend(test_index)
  gold.extend(test_df['labels'])
  text.extend(test_df['text'])

  model_bert = ClassificationModel('bert', 'bert-base-cased', args=model_bert_args)
  
  model_bert.train_model(train_df)
  predictions_bert, prob_bert = model_bert.predict(test_df.text.to_list())
  pred_bert.extend(predictions_bert)

  model_roberta = ClassificationModel("roberta", "roberta-base", args=model_roberta_args)

  model_roberta.train_model(train_df)
  predictions_roberta, prob_roberta = model_roberta.predict(test_df.text.to_list())
  pred_roberta.extend(predictions_roberta)

  model_hatebert = ClassificationModel("bert", "GroNLP/hateBERT", args=model_hatebert_args)

  model_hatebert.train_model(train_df)
  predictions_hatebert, prob_hatebert = model_roberta.predict(test_df.text.to_list())
  pred_hatebert.extend(predictions_hatebert)



NameError: ignored

In [ ]:
output = pd.DataFrame(columns = ['text', 'bert', 'roberta', 'hatebert', 'label']) 
output['text'] = text
output['bert'] = pred_bert
output['roberta'] = pred_roberta
output['hatebert'] = pred_hatebert
output['label'] = gold
print(output)

IndexError: ignored

In [ ]:
#Add features 

#Add number of characters of each sentence
num_char = [len(i) for i in output['text']]
output['num_char'] = num_char

num_words = [len(i.split()) for i in output['text']]
output['num_words'] = num_words
output

F1 score for Bert model: 0.16713091922005568
F1 score for Roberta model: 0.16713091922005568
F1 score for Hatebert model: 0.16713091922005568


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

X_train = output.loc[:, output.columns != 'label']
Y_train = output['label']
X_test = df2.loc[:, df2.columns != 'labels']
Y_test = df2['labels']

text_clf= Pipeline([
    ('vect',CountVectorizer()),
    ('clf', MLPClassifier(max_iter=300))])

clf = text_clf.fit(X_train, Y_train)

predictions = text_clf.predict(X_test).tolist()

print(classification_report(predictions,Y_test))

CROSS DOMAIN STACKING ENSEMBLE

In [ ]:
train = df3

rskf = StratifiedKFold(n_splits=5, shuffle=True)

pred_bert = []
pred_roberta = []
pred_hatebert = []

idx = []
gold = []
text = []

for train_index, test_index in rskf.split(train['text'], train['labels']):

  train_df = train.iloc[train_index]
  test_df = train.iloc[test_index]

  idx.extend(test_index)
  gold.extend(test_df['labels'])
  text.extend(test_df['text'])

  model_bert = ClassificationModel('bert', 'bert-base-cased', args=model_bert_args)
  
  model_bert.train_model(train_df)
  predictions_bert, prob_bert = model_bert.predict(test_df.text.to_list())
  pred_bert.extend(predictions_bert)

  model_roberta = ClassificationModel("roberta", "roberta-base", args=model_roberta_args)

  model_roberta.train_model(train_df)
  predictions_roberta, prob_roberta = model_roberta.predict(test_df.text.to_list())
  pred_roberta.extend(predictions_roberta)

  model_hatebert = ClassificationModel("bert", "GroNLP/hateBERT", args=model_hatebert_args)

  model_hatebert.train_model(train_df)
  predictions_hatebert, prob_hatebert = model_roberta.predict(test_df.text.to_list())
  pred_hatebert.extend(predictions_hatebert)



In [ ]:
output = pd.DataFrame(columns = ['text', 'bert', 'roberta', 'hatebert', 'label']) 
output['text'] = text
output['bert'] = pred_bert
output['roberta'] = pred_roberta
output['hatebert'] = pred_hatebert
output['label'] = gold
print(output)

In [ ]:
#Add features 

#Add number of characters of each sentence
num_char = [len(i) for i in output['text']]
output['num_char'] = num_char

num_words = [len(i.split()) for i in output['text']]
output['num_words'] = num_words
output

In [ ]:
X_train = output.loc[:, output.columns != 'label']
Y_train = output['label']
X_test = df2.loc[:, df2.columns != 'labels']
Y_test = df2['labels']

text_clf= Pipeline([
    ('vect',CountVectorizer()),
    ('clf', MLPClassifier(max_iter=300))])

clf = text_clf.fit(X_train, Y_train)

predictions = text_clf.predict(X_test).tolist()

print(classification_report(predictions,Y_test))

###ERROR ANALYSIS part

Here we can take a look at the different wrong predictions that have been made and analyse the causes that could explains these misunderstandings.



In [ ]:
wrong_predictions_Bert_4

In [ ]:
wrong_predictions_Roberta

In [ ]:
wrong_predictions_HateBert

###Cross-domain experiments : 
We train the BERT (pre-trained language model), RoBERTa (pre-trained language model) and HateBERT (re-trained transformer model) on the HASOC Train dataset (olid-train-small.csv) and evaluate them on the OLID test set (olid_test.csv)

In [ ]:
#Bert
#We train the Bert model on the other dataset (Cross Domain).
model_Bert_4.train_model(df3, output_dir="Bert CD Training")

#And evaluate it on the first dataset
result_Bert_CD, model_outputs_Bert_CD, wrong_predictions_Bert_CD = model_Bert_4.eval_model(df_val)

#Roberta
#training
model_Roberta.train_model(df3, output_dir="Roberta CD Training")
#eval
result_Roberta_CD, model_outputs_Roberta_CD, wrong_predictions_Roberta_CD = model_Roberta.eval_model(df_val)

#HateBert
#training
model_HateBert.train_model(df3, output_dir="HateBert CD Training")
#eval
result_HateBert_CD, model_outputs_HateBert_CD, wrong_predictions_HateBert_CD = model_HateBert.eval_model(df_val)

In [ ]:
F1_Bert_CD = 2*((result_Bert_CD["tp"] / (result_Bert_CD["tp"] + result_Bert_CD["fp"])) * (result_Bert_CD["tp"]/(result_Bert_CD["tp"]+result_Bert_CD["fn"]))) / ((result_Bert_CD["tp"] / (result_Bert_CD["tp"] + result_Bert_CD["fp"])) + (result_Bert_CD["tp"]/(result_Bert_CD["tp"]+result_Bert_CD["fn"])))
F1_Roberta_CD = 2*((result_Roberta_CD["tp"] / (result_Roberta_CD["tp"] + result_Roberta_CD["fp"])) * (result_Roberta_CD["tp"]/(result_Roberta_CD["tp"]+result_Roberta_CD["fn"]))) / ((result_Roberta_CD["tp"] / (result_Roberta_CD["tp"] + result_Roberta_CD["fp"])) + (result_Roberta_CD["tp"]/(result_Roberta_CD["tp"]+result_Roberta_CD["fn"])))
F1_HateBert_CD = 2*((result_HateBert_CD["tp"] / (result_HateBert_CD["tp"] + result_HateBert_CD["fp"])) * (result_HateBert_CD["tp"]/(result_HateBert_CD["tp"]+result_HateBert_CD["fn"]))) / ((result_HateBert_CD["tp"] / (result_HateBert_CD["tp"] + result_HateBert_CD["fp"])) + (result_HateBert_CD["tp"]/(result_HateBert_CD["tp"]+result_HateBert_CD["fn"])))

###The results

In [ ]:
print ("BERT MODEL : ")
print(result_Bert_CD)
print((result_Bert_CD["tp"] / (result_Bert_CD["tp"] + result_Bert_CD["fp"])))
print((result_Bert_CD["tp"]/(result_Bert_CD["tp"]+result_Bert_CD["fn"])))
print(F1_Bert_CD)

In [ ]:
print ("RoBERTa MODEL : ")
print(result_Roberta_CD)
print((result_Roberta_CD["tp"] / (result_Roberta_CD["tp"] + result_Roberta_CD["fp"])))
print((result_Roberta_CD["tp"]/(result_Roberta_CD["tp"]+result_Roberta_CD["fn"])))
print(F1_Roberta_CD)

In [ ]:
print ("HateBERT MODEL : ")
print(result_HateBert_CD)
print((result_HateBert_CD["tp"] / (result_HateBert_CD["tp"] + result_HateBert_CD["fp"])))
print((result_HateBert_CD["tp"]/(result_HateBert_CD["tp"]+result_HateBert_CD["fn"])))
print(F1_HateBert_CD)

#Error analysis
 Analysis of the wrong predictions by the three models in Cross-domain experiment.

In [ ]:
wrong_predictions_Bert_CD

In [ ]:
wrong_predictions_Roberta_CD

In [ ]:
wrong_predictions_HateBert_CD

##Other models experiment 

We will now explore other techniques such as CNNs, LSTMs and BiLSTMs to compare.

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.pipeline import Pipeline
traindata=pd.read_csv (r'olid-train-small.csv')
train_x=traindata['text'].tolist()
train_y=traindata['labels'].tolist()
testdata=pd.read_csv (r'olid-test.csv')
test_x=testdata['text'].tolist()
test_y=testdata['labels'].tolist()

text_clf= Pipeline([
    ('vect',CountVectorizer()),
    ('clf', MLPClassifier(max_iter=300))])

clf = text_clf.fit(train_x, train_y)


In [ ]:

predictions = text_clf.predict(test_x).tolist()

print(classification_report(predictions,test_y))

## Other models on cross domain data


In [ ]:
traindata=pd.read_csv (r'hasoc-train.csv')
train_x=traindata['text'].tolist()
train_y=traindata['labels'].tolist()
testdata=pd.read_csv (r'olid-test.csv')
test_x=testdata['text'].tolist()
test_y=testdata['labels'].tolist()

text_clf= Pipeline([
    ('vect',CountVectorizer()),
    ('clf', MLPClassifier(max_iter=300))])

clf2 = text_clf.fit(train_x, train_y)

In [ ]:
predictions2 = text_clf.predict(test_x).tolist()

print(classification_report(predictions2,test_y))